In [1]:
import os

In [2]:
%pwd

'/Users/kanayojustice/Documents/Data_scientist_projects/AutoPredict/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/kanayojustice/Documents/Data_scientist_projects/AutoPredict'

In [5]:
#Update the entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    Local_data_file: Path
    unzip_dir: Path

In [6]:
#Update the configuration manager in src config
from Autopredictor.src.constants import *
from Autopredictor.src.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            Local_data_file=config.Local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
#update components
import os
import urllib.request as request
import zipfile
from Autopredictor.src.logging import logger
from Autopredictor.src.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.Local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.Local_data_file
            )
            logger.info(f"{filename} Download! with the following info: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.Local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.Local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
#pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-30 13:59:15,552: INFO:common: yaml file: /Users/kanayojustice/Documents/Data_scientist_projects/AutoPredict/config/config.yaml loaded successfully]
[2024-05-30 13:59:15,558: INFO:common: yaml file: /Users/kanayojustice/Documents/Data_scientist_projects/AutoPredict/params.yaml loaded successfully]
[2024-05-30 13:59:15,565: INFO:common: yaml file: /Users/kanayojustice/Documents/Data_scientist_projects/AutoPredict/schema.yaml loaded successfully]
[2024-05-30 13:59:15,566: INFO:common: created directory at: artifacts]
[2024-05-30 13:59:15,568: INFO:common: created directory at: artifacts/data_ingestion]


[2024-05-30 13:59:17,094: INFO:1347631616: artifacts/data_ingestion/autopredict_cleaned_data.zip Download! with the following info: 
Connection: close
Content-Length: 1268268
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "5d493dbc444db2fa89e3a3d034d18ca7eb909fbdea2a0399f3081049b971949a"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F16C:35B3ED:341B76:3FCA50:66587823
Accept-Ranges: bytes
Date: Thu, 30 May 2024 12:59:16 GMT
Via: 1.1 varnish
X-Served-By: cache-lhr-egll1980061-LHR
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1717073956.415071,VS0,VE203
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: d12e1555e47f8062172697dfe5b4ce662d305dbd
Expires: Thu, 30 May 2024 13:04:16 GMT
Source-Age: 0

]
